In [1]:
from bs4 import BeautifulSoup
import urllib.request
import csv
import pickle
from urllib.error import HTTPError
from collections import defaultdict
import html5lib

In [18]:
def getSeasonScheduleSoup(address):
    try:
        with urllib.request.urlopen(address) as url:
            s = url.read()
            bs = BeautifulSoup(s,"lxml")
            return bs.findAll('table')[0]
    except HTTPError:
        return None
    return None

In [19]:
g = getSeasonScheduleSoup(address)

In [89]:

def getGameLinksFromYear(year):
    
    address = 'http://www.basketball-reference.com/leagues/NBA_'+year+'_games-'
    months = ['november','december','january','february','march','april','may','june']
    links = []
    for month in [address+m+'.html' for m in months]:
        games = getSeasonScheduleSoup(month)
        if not games:
            continue
        for body in games.findAll('tbody'):
            for row in body.findAll('tr'):
        #         print(row)
        #         print(h)
        #         print('\n')
                h = row.findAll('th')
                try:
                    links.append(h[0]['csk']) # unique identifier for the game played(year,month,day,home team)
                except :# in case season does not work
                    print('ERROR: ',h)
    return links

#getGameLinksFromYear('2006')

In [71]:
def getYearsInRange(st,ed):
    
    seasons = {}
    for year in range(st,ed):
        print(year)
        seasons[str(year)] = getGameLinksFromYear(str(year))
    
    
    out = open("SeasonGames_"+str(st)+'-'+str(ed)+".pkl","wb")
    pickle.dump(seasons, out)

In [92]:
#getYearsInRange(1985,2017)


In [47]:
seasons = pickle.load(open('SeasonGames_1985-2017.pkl','rb'))

In [142]:
def readBoxScore(game):
    address = 'http://www.basketball-reference.com/boxscores/'+game+'.html'
    try:
        with urllib.request.urlopen(address) as url:
            s = url.read()
            bs = BeautifulSoup(s,"lxml")
            #return bs
    except HTTPError:
        return None
    data = []
    for t in bs.findAll('table'):
    # should go away basic, away advanced,home basic, home advanced
        data.append([a.text for a in t.findAll('tr')[-1].find_all('td')])
    return data


In [126]:
print(testing)
# data = []
# for t in testing.findAll('table'):
#     # should go away basic, away advanced,home basic, home advanced
    
#     total = [a.text for a in t.findAll('tr')[-1].find_all('td')]
    

[['240', '32', '78', '.410', '1', '3', '.333', '18', '30', '.600', '13', '25', '38', '15', '11', '0', '17', '31', '83'], ['240', '.455', '.417', '.038', '.385', '31.7', '71.4', '50.0', '46.9', '12.2', '0.0', '15.7', '100.0', '91.7', '110.5'], ['240', '33', '71', '.465', '2', '10', '.200', '32', '43', '.744', '10', '28', '38', '16', '8', '8', '13', '25', '100'], ['240', '.556', '.479', '.141', '.606', '28.6', '68.3', '50.0', '48.5', '8.8', '10.7', '12.6', '100.0', '110.5', '91.7']]


In [ ]:
results = {s:dict() for s in seasons}
errors = []
for season in seasons:
    print(season)
    for game in seasons[season]:
        try:
            results[season][game] = htmlTable2Pandas(game)
        except:
            print(game)
            errors.append(game)
            break
            
out = open("boxScores_1985-2017","wb")
pickle.dump(results, out)

1997
199611010BOS
2000
199911020CHH
1994
199311050ATL
2005


In [53]:
def htmlTable2Pandas(game):
    address = 'http://www.basketball-reference.com/boxscores/'+game+'.html'
    try:
        with urllib.request.urlopen(address) as url:
            s = url.read()
            bs = BeautifulSoup(s,"lxml")
            #return bs
    except HTTPError:
        return None
    frames = []
    for t in bs.findAll('table'):
        data = []
        for row in t.findAll('tr'):
            player = ''
            try:
                player = row.findAll('th')[0]['data-append-csv'] #should be id of player
            except: 
                player = None
            if not player:
                continue
            vals = [player]+[r.text for r in row.findAll('td')]
            data.append(vals)
        df = pd.DataFrame(data)#columns = ['NAME','MP','FG','FGA','FGP','3P','3PA','3PP','FT','FTA','FTP','ORB','DRB','TRB','AST,STL'])
        h,w = df.shape
        if h<6 or w<6:
            continue
        frames.append(df)
    # should go away basic, away advanced,home basic, home advanced
    if len(frames)!=4:return
    frames[0].columns = ['NAME','MP','FG','FGA','FGP','3P','3PA','3PP','FT','FTA','FTP','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PM']
    frames[2].columns = ['NAME','MP','FG','FGA','FGP','3P','3PA','3PP','FT','FTA','FTP','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PM']
    
    frames[1].columns = ['NAME','MP','TSP','eFGP','3PAr','FTr','ORBP','DRBP','TRBP','ASTP','STLP','BLKP','TOVP','USGP','ORtg','DRtg']
    frames[3].columns = ['NAME','MP','TSP','eFGP','3PAr','FTr','ORBP','DRBP','TRBP','ASTP','STLP','BLKP','TOVP','USGP','ORtg','DRtg']
    return frames



In [2]:
import pandas as pd

NameError: name 'seasons' is not defined